<a href="https://colab.research.google.com/github/c444roline/dxc-ai-maintenance-assistant/blob/main/cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#leading csv files in, mounting drive
import pandas as pd
from google.colab import files,drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)  # Ensure Google Drive is mounted

# Define the path to the CSV files in your Google Drive
pet_bottle_faults_path = '/content/drive/MyDrive/pet_bottle_faults.csv'
stressesinpebottles_path = '/content/drive/MyDrive/stressesinpebottles2.csv'

# Load the CSV files into pandas DataFrames
df_1 = pd.read_csv(pet_bottle_faults_path) # Use the correct path variable
print("pet_bottle_faults.csv loaded successfully.")

df_2 = pd.read_csv(stressesinpebottles_path)
print("updated_stressesinpebottles.csv loaded successfully.")

Mounted at /content/drive
pet_bottle_faults.csv loaded successfully.
updated_stressesinpebottles.csv loaded successfully.


In [ ]:
#cleaned pet_bottle_faults file!
import re

# Extract the contents for the first column
contents_column = df_1.iloc[3:32, 0].tolist()  # Extract rows 3 to 32

# Create an empty list to store the data for the other columns
data = []

# Helper function to identify if a line is a content title
def is_content_title(text):
    cleaned_text = re.sub(r"^[^\w\s]*\d*\.\s*", "", text.strip())  # Remove leading symbols and numbers
    return cleaned_text.isupper()

# Iterate through the DataFrame to extract data for other columns
index = 32
for i in range(32, len(df_1)):
    try:
        description, cause, solution = [], [], []

        # Description Extraction
        while index < len(df_1) and not any(desc in df_1.iloc[index, 0] for desc in ["DESCRIPTION:", "DESCRIPTION/FAULT:"]):
            index += 1
        if index < len(df_1):
            index += 1  # Move to the next line
            while index < len(df_1) and "CAUSE:" not in df_1.iloc[index, 0] and "SOLUTION:" not in df_1.iloc[index, 0] and not is_content_title(df_1.iloc[index, 0]):
                description.append(df_1.iloc[index, 0])
                index += 1

        # Cause Extraction
        while index < len(df_1) and "CAUSE:" not in df_1.iloc[index, 0]:
            index += 1
        if index < len(df_1):
            index += 1  # Move to the next line
            while index < len(df_1) and "SOLUTION:" not in df_1.iloc[index, 0] and not is_content_title(df_1.iloc[index, 0]):
                cause.append(df_1.iloc[index, 0])
                index += 1

        # Solution Extraction
        while index < len(df_1) and "SOLUTION:" not in df_1.iloc[index, 0]:
            index += 1
        if index < len(df_1):
            index += 1  # Move to the next line
            while index < len(df_1) and not is_content_title(df_1.iloc[index, 0]):
                solution.append(df_1.iloc[index, 0])
                index += 1

        # Combine the solution lines into a single text
        combined_solution = "\n".join(solution).strip()

        # Format the solution correctly - finalize solution formatting
        formatted_solution = reformat_solution(combined_solution)

        # Create a row with the gathered information
        row = {
            "description": "\n".join(description).strip(),
            "cause": "\n".join(cause).strip(),
            "solution": formatted_solution,
        }
        data.append(row)

    except IndexError:
        # Handle cases where there might be incomplete rows at the end
        break

# Create a DataFrame from the data for other columns
other_columns_df = pd.DataFrame(data)

# Use your original logic to extract and clean the contents column
def remove_leading_numbers(text):
    text = str(text)  # Convert to string in case of mixed data types
    text = re.sub(r"^\d+\.\s*", "", text)  # Remove leading numbers and period with optional whitespace
    return text

# Apply the function to the specified columns
contents_column = [remove_leading_numbers(content) for content in contents_column]

# Define a function to clean the contents column
def clean_contents(text):
    text = str(text)  # Convert to string in case of mixed data types
    match = re.search(r"([A-Z\s]+\([A-Z]+\))", text)  # Find uppercase words with parentheses
    if match:
        return match.group(1).strip()  # Return the matched part
    else:
        # If no parentheses, remove periods and everything after
        text = re.sub(r"\.+.*", "", text).strip()
        return text

# Clean the extracted contents
contents_column = [clean_contents(content) for content in contents_column]

# Combine the contents column and other columns DataFrame
structured_df = pd.DataFrame({"contents": contents_column}).join(other_columns_df)

# Save the structured DataFrame to a CSV file in Google Drive
file_path = '/content/drive/My Drive/organized_pet_bottle_faults.csv'
structured_df.to_csv(file_path, index=False)

# Display the structured DataFrame
display(structured_df)

print(f"CSV file saved successfully at: {file_path}")

,contents,description,cause,solution
0,BLOWN FINISH,"Expansion of the finish, located at or above t...",1. Oven ambient temperature too high.\n2. Stan...,1. Increase oven ventilation or direct air on ...
1,SEALING SURFACE AND NECK SUPPORT RING DAMAGE,As the transfer arm releases the preform into ...,1. Poor transfer arm positioning causing misal...,1. Re-align transfer arm — height and timing.\...
2,NECK CONSTRICTION,A fold of material in the neck or upper should...,"1. Low pre-blow, delayed or missing.\n2. Flow ...","1. If problem is occurring on all stations, mo..."
3,PERPENDICULARITY,The distortion may occur at or above the neck ...,1. Mis-alignment of the preform to the mould o...,1. Re-align transfer arm (height and timing). ...
4,HARD SHOULDER,An excess of material remains in the neck and ...,1. Incorrect heat profile.\n2. Incorrect blowi...,1. Reduce heat in the body and/or base before ...
5,THIN SHOULDER,"Shoulder wall thickness too thin, high base we...",1. Blowing parameters require adjustment.\n2. ...,1. Increase pre-blow pressure.\n2. Move pre-bl...
6,OPALESCENCE (TOO WARM,"Hazy-white in colour, usually found on the out...",1. Preform set point too high.\n2. Too much he...,1. Reduce set point temperature.\n2. Reduce he...
7,EXCESSIVE FILL POINT DROP,"When carbonated, the bottle appears partially ...",1. The overall temperature of the preform is t...,1. Reduce overall preform temperature allowing...
8,PEARLESCENCE (TOO COLD),Blue in colour appearing fragmented and patchy...,Pearlescence results from stretching of molecu...,1. Increase overall preform temperature — set ...
9,FLAT SIDES,Flat sides are usually found at the vertical p...,1. Feet not blown due to a low main blow.\n2. ...,1. Check compensation pressure on moulds.\n2. ...


CSV file saved successfully at: /content/drive/My Drive/organized_pet_bottle_faults.csv


In [ ]:
#cleaned stressesinpebottles file!

import pandas as pd
import re

def extract_sections(df):
    data = []
    current_section = None
    current_text = ""
    abstract_text = ""
    figures_text = ""
    figures_pattern = re.compile(r"^Fig\.\s*\d+\.")  # Regex to detect figure descriptions starting with "Fig."

    # Extract the abstract text starting from row 7 until "INTRODUCTION"
    for i in range(7, len(df)):
        cell_value = str(df.iloc[i, 0]).strip()

        # Stop at "INTRODUCTION" and set aside the abstract
        if cell_value.upper() == "INTRODUCTION":
            data.append(["ABSTRACT", abstract_text.strip()])
            break

        abstract_text += cell_value + '\n'

    # Continue extracting the rest of the sections as usual
    for i in range(4, len(df)):
        cell_value = str(df.iloc[i, 0]).strip()

        # Check for known section headers
        if cell_value.upper() in ['INTRODUCTION', 'EXPERIMENTAL', 'RESULTS AND DISCUSSION', 'CONCLUSIONS', 'REFERENCES']:
            # Append the accumulated data if switching to a new section
            if current_section:
                data.append([current_section, current_text.strip()])

            # Check if the current section is REFERENCES
            if cell_value.upper() == 'REFERENCES':
                # Add FIGURES section if there are collected figures
                if figures_text:
                    data.append(['FIGURES', figures_text.strip()])
                current_section = cell_value.upper()
                current_text = ""  # Reset text for REFERENCES
            else:
                # Reset for the new section
                current_section = cell_value.upper()
                current_text = ""
        else:
            # Remove figure descriptions from sections and collect them separately
            if figures_pattern.match(cell_value):
                figures_text += cell_value + '\n'
            else:
                current_text += cell_value + '\n'

    # Collect any text below the REFERENCES section
    if current_section and current_section.upper() == 'REFERENCES':
        references_text = []  # List to hold the formatted references
        point_number = 1  # Start point numbering
        current_reference = ""  # To accumulate lines for the current reference

        for j in range(i + 1, len(df)):  # Start from the next row after the last processed row
            cell_value = str(df.iloc[j, 0]).strip()
            if cell_value:  # Check for non-empty lines
                # Check if the line starts with a point number (e.g., "1. ", "2. ")
                if cell_value[0].isdigit() and '.' in cell_value:
                    # If we have accumulated text for the previous reference, save it
                    if current_reference:
                        references_text.append(f"{point_number}. {current_reference.strip()}")
                        point_number += 1
                        current_reference = ""  # Reset for the new reference

                # Accumulate the current reference line
                current_reference += cell_value + ' '  # Append a space to separate lines

        # Add the last accumulated reference if exists
        if current_reference:
            references_text.append(f"{point_number}. {current_reference.strip()}")

        # Join the formatted references with new lines
        current_text += '\n'.join(references_text)

        # Append the last section (REFERENCES)
        data.append([current_section, current_text.strip()])

    # Create and return the DataFrame
    return pd.DataFrame(data, columns=['section', 'text'])

# Apply the function to the dataframe
new_df = extract_sections(df_2)

# Display the new structured DataFrame
display(new_df)

# Save the new structured DataFrame to Google Drive as a CSV file
file_path = '/content/drive/MyDrive/organized_stressesinpebottles.csv'  # Change the path if needed
new_df.to_csv(file_path, index=False)

# Display confirmation message
print(f"File saved to {file_path}")


,section,text
0,ABSTRACT,"A commercial blow-molding grade, high-density ..."
1,INTRODUCTION,Commercial bottles must satisfy performance cr...
2,EXPERIMENTAL,"A commercial blow-molding grade, high-density\..."
3,RESULTS AND DISCUSSION,A typical three-dimensional thickness distribu...
4,CONCLUSIONS,"The results of this study suggest that, in the..."
5,FIGURES,Fig. 1. Shear viscosity behavior of Resin D.\n...
6,REFERENCES,"1. H. Frank and W. Wengler, Verpack Runds, 22,..."


File saved to /content/drive/MyDrive/organized_stressesinpebottles.csv
